# Notebook for Creating Figures from OpenFOAM Solutions

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import pyvista as pv

## Define Helper Files

In [14]:
def camera_settings(PYLON=False):
    """ Define settings for viewing the figures """
    if PYLON is True:
        camera = pv.Camera()
        camera.position = (15.9143, 3.83627, 28.3582)
        camera.focal_point = (15.9143, 3.83627, 13.75)
        camera.view_angle = 30
    else:
        camera = pv.Camera()
        camera.position = (15.9919, 3.10319, 67.1243)
        camera.focal_point = (15.9919, 3.10319, -5.08432e-08)
        camera.view_angle = 30
    
    return camera

In [4]:
def slice_center(mesh, NORMAL='z', ORIGIN=[0, 0, 0], translate=False, CONTOUR=False):
    """Slice mesh through center in normal direction, move to zero normal."""
    slice_mesh = mesh.slice(normal=NORMAL, origin=ORIGIN, contour=CONTOUR)
    if translate is not False:
        if NORMAL == 'z':
            slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
        elif NORMAL == 'x':
            slice_mesh.translate((-slice.mesh.center[0], 0, 0), inplace=True)
        elif NORMAL == 'y':
            slice_mesh.translate((0, -slice.mesh.center[1], 0), inplace=True)
        else:
            print('ERROR: Normal not found - ', NORMAL)
            slice_mesh = None
    return slice_mesh

In [5]:
def read_mesh(nfile):
    """ Read the OpenFOAM solution, needs the file generated by paraFoam (e.g. *.OpenFOAM) """
    # Read the reference file created by openFOAM `parafoam`
    reader = pv.POpenFOAMReader(nfile)
    # set the active time 
    reader.set_active_time_value(reader.time_values[-1])
    reader.cell_to_point_creation = True
    # Define the mesh
    mesh = reader.read()
    # Define the internal mesh and boundaries
    internal = mesh["internalMesh"]
    boundaries = mesh["boundary"]
    
    return internal, boundaries

In [31]:
def plot_field(ndir, 
               nfile,
               FIELD='U', 
               RANGE=[95, 145], 
               CONTOUR=True, 
               ORIGIN=[0, 0, 0], 
               NORMAL='z', 
               CMAP='bwr',
               SAVE=False,
               PYLON=False):
    """ 
    Create 2-D Figure of the OpenFOAM internal mesh for selected field. 
    Normal decides which plane to exclude from display.
    Origin moves where the slice is preformed.
    """
    # add extension to the input directory file to reference *.OpenFOAM files
    solution = ndir + nfile['filename'] + nfile['filename'][:-1] + '.OpenFoam'
    
    # Define label for input field
    LABEL = {"U" : "Velocity [m/s]",
             "p" : "Pressure [Pa]", 
             "T" : "Temperature [C]"
            }
    
    # Read the Mesh
    internal, boundaries = read_mesh(solution)
    
    # Slice the Mesh
    if PYLON is True:
        p3_slice = slice_center(internal, NORMAL=NORMAL, ORIGIN=nfile['pylon'])
    else:
        p3_slice = slice_center(internal, NORMAL=NORMAL, ORIGIN=ORIGIN)
    
    # Contour the Mesh
    contours = p3_slice.contour(scalars=FIELD, isosurfaces=15, rng=RANGE)
    
    # Set a custom position and size
    sargs = dict(height=0.10, 
                 vertical=False, 
                 position_x=0.2, 
                 position_y=0.05, 
                 fmt="%.0f", 
                 color='black', 
                 n_labels=6, 
                 title=LABEL[FIELD])
    
    # Create the plotter
    pt = pv.Plotter()
    # Add the slice mesh
    pt.add_mesh(p3_slice, 
                scalars=FIELD, 
                preference='point', 
                cmap=CMAP, 
                clim=RANGE,
                scalar_bar_args=sargs)
    
    # Add the contours
    pt.add_mesh(contours, 
                color='white', 
                preference='cell', 
                line_width=1)
    
    # Add a ruler
    if PYLON is True:
        pt.add_ruler(pointa=[10, -0.35, 0.0],
                     pointb=[20, -0.35, 0.0],
                     title='Distance [m]',
                     tick_color='black',
                     label_color='black'
                    )
    else:
        pt.add_ruler(pointa=[-5, -5, 0.0],
                     pointb=[35, -5, 0.0],
                     title='Distance [m]',
                     tick_color='black',
                     label_color='black'
                    )
   
    # Add a title
    pt.add_title(nfile['filename'][:-1], 
                 font='courier', 
                 color='k', 
                 font_size=10
                )
    
    # Define the camera view
    if PYLON is True:
        pt.camera = camera_settings(PYLON=True)
    else:
        pt.camera = camera_settings()
    
    # Save the figure
    if SAVE is True:
        # Define a figure title
        nout = nfile['filename'][:-1] + '_' + str(FIELD) + '.svg'
        # Save the figure
        pt.save_graphic(nout, raster=True)
        
    # Display the figure
    pt.show()

## Define the Directory, Case Structure, Fields and Pylon Locations

In [7]:
# Define the directory hosting the solution directories
solution_dir = "/Users/jrobrien/Dissertation/data/solutions/"
# Define available solution directories
case_dir = {'navy_120' : {'filename' : 'NASA_navyPylon_v2_tas120_aoa0_900T33/',
                          'pylon' : [0, 2.74, 13.75]
                         },
            'extend_120' : {'filename' : 'NASA_extendedPylon_v2_tas120_aoa0_900T33/',
                            'pylon' : [0, 3.05, 13.7]
                           },
            'no_120' : {'filename' : 'NASA_noPylons_v2_tas120_aoa0_900T33/'}
           }

In [8]:
scalar_field = {'U' : {'range' : [95, 145]},
                'T' : {'range' : [295, 318]},
                'p' : {'range' : [89000, 91000]}
               }

## Process the OpenFOAM Solutions

In [34]:
# Attempt at Processing this way
for case in case_dir:
    if case == 'extend_120':
        print(case_dir[case])
        #plot_field(solution_dir, case_dir[case], FIELD='p', RANGE=[89000, 91000], SAVE=True, PYLON=True)
        plot_field(solution_dir, case_dir[case], SAVE=True, PYLON=True)

{'filename': 'NASA_extendedPylon_v2_tas120_aoa0_900T33/', 'pylon': [0, 3.05, 13.7]}


Widget(value="<iframe src='http://localhost:53791/index.html?ui=P_0x31d2e5520_15&reconnect=auto' style='width:…